In [1]:
import requests
import pandas as pd
import time

def get_usdc_pairs():
    resp = requests.get('https://api.kraken.com/0/public/AssetPairs').json()
    pairs = []
    for key, val in resp['result'].items():
        if val.get('wsname') and '/USDC' in val['wsname'] and '.d' not in key:
            pairs.append({'pair': key, 'wsname': val['wsname'], 'altname': val['altname']})
    return pairs

def get_ticker(pair):
    resp = requests.get(f"https://api.kraken.com/0/public/Ticker?pair={pair}").json()
    result = list(resp['result'].values())[0]
    bid = float(result['b'][0])
    ask = float(result['a'][0])
    spread = (ask - bid) / ((ask + bid) / 2) * 100
    volume_24h = float(result['v'][1])
    avg_price_24h = float(result['p'][1])
    return bid, ask, spread, volume_24h, avg_price_24h

def get_trades(altname):
    resp = requests.get(f"https://api.kraken.com/0/public/Trades?pair={altname}").json()
    trades = list(resp['result'].values())[0]
    return trades

def compute_hourly_volume_usdc(trades):
    now = time.time()
    since = now - 3600
    trades_last_hour = [t for t in trades if float(t[2]) > since]
    vol = sum(float(t[1]) for t in trades_last_hour)
    # Средняя цена за час
    avg_price = (sum(float(t[0]) * float(t[1]) for t in trades_last_hour) / vol) if vol > 0 else 0
    vol_usdc = vol * avg_price
    ntrades = len(trades_last_hour)
    return ntrades, vol, avg_price, vol_usdc

usdc_pairs = get_usdc_pairs()
print(f"Всего найдено {len(usdc_pairs)} пар к USDC на Kraken.")

summary = []
for p in usdc_pairs:
    try:
        bid, ask, spread, vol24, avg_price_24h = get_ticker(p['pair'])
        # Фильтруем по spread < 0.2%
        if spread >= 0.2:
            continue
        # 24h volume в USDC
        vol24_usdc = vol24 * avg_price_24h
        if vol24_usdc < 100000:
            continue
        trades = get_trades(p['altname'])
        ntrades, vol_hour, avg_price_hour, vol_hour_usdc = compute_hourly_volume_usdc(trades)
        summary.append({
            'wsname': p['wsname'],
            'spread_%': spread,
            'volume_24h': vol24,
            'volume_24h_usdc': vol24_usdc,
            'trades_last_hour': ntrades,
            'real_volume_last_hour': vol_hour,
            'avg_price_last_hour': avg_price_hour,
            'real_volume_last_hour_usdc': vol_hour_usdc
        })
    except Exception as e:
        print(f"Ошибка с парой {p['wsname']}: {e}")

df = pd.DataFrame(summary)
if df.empty:
    print("Нет подходящих пар по всем критериям!")
else:
    df = df.sort_values('real_volume_last_hour_usdc', ascending=False)
    top3 = df.head(3)[[
        'wsname',
        'spread_%',
        'volume_24h_usdc',
        'trades_last_hour',
        'real_volume_last_hour_usdc',
        'avg_price_last_hour'
    ]]
    print("=== Топ-3 пары по реальному объёму за последний час (в USDC) ===")
    display(top3)


Всего найдено 38 пар к USDC на Kraken.
=== Топ-3 пары по реальному объёму за последний час (в USDC) ===


,wsname,spread_%,volume_24h_usdc,trades_last_hour,real_volume_last_hour_usdc,avg_price_last_hour
4,XBT/USDC,0.000009,7.331189e+06,406,627475.489830,105761.573148
1,ETH/USDC,0.000390,2.862704e+06,135,248988.857092,2567.438875
5,XDG/USDC,0.140812,5.402394e+05,68,154995.026902,0.233700
